In [1]:
import os 
import random 
from datetime import date
from collections import defaultdict
from tqdm import tqdm
from copy import deepcopy
import numpy as np 
import pandas as pd

import torch 
from torch import nn, Tensor
from torch.nn import functional as F
from torch.nn.modules import MultiheadAttention, Linear, Dropout, BatchNorm1d, TransformerEncoderLayer
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils import weight_norm
from torch.optim import Adam, Adagrad
from torch.nn import MSELoss

from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import torchinfo 
from pytorch_metric_learning.losses import ContrastiveLoss
from pytorch_metric_learning.distances import LpDistance, CosineSimilarity

In [2]:
from src.datasets.data import PAMAP2Reader
# from src.datasets.dataset import PAMAP2Dataset

from src.models.ts_transformer import RelativeGlobalAttention
from src.models.loss import FeatureLoss, AttributeLoss 
from src.utils.losses import SupConLoss

from src.utils.analysis import action_evaluator

In [3]:
from sklearn.manifold import TSNE
from umap import UMAP

import matplotlib.pyplot as plt 
import plotly.express as px

In [4]:
# setup model configurations
config = {
    # general information
    "experiment-name": "test-001", 
    "datetime": date.today(),
    "device": "gpu",
    "dataset": "PAMAP2", # "PAMAP2", "DaLiAc", "UTD"
    "Model": "RelativeGlobalAttention",
    "sem-space": 'attr',
    # model training configs
    "include_attribute_loss": True, 
    "semantic_size": 128,
    "n_actions": 18,
    "folding": True,
    "lr": 0.001,
    'imu_lr': 0.0001,
    'ae_lr': 0.005,
    'imu_alpha': 0.6,
    'ae_alpha': 0.7,
    "n_epochs": 5,
    "optimizer": "adam",
    "batch_size": 16,
    "semantic_loss": "cosine_distance",
    # model configs
    "d_model": 128, 
    "num_heads": 2,
    "feat_size": 64, # skel-AE hidden size and IMU-Anc output size
    # dataset configs
    "window_size": 5, 
    "overlap": 0.5,
    "seq_len": 50,  # skeleton seq. length
    "seen_split": 0.2,
    "unseen_split": 0.8
}

---
Init Model 

In [5]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)
    
class TCNEnc(nn.Module):
    def __init__(self, seq_len=50,num_features=36, num_channels=[20,20,20,4], embedding_size=128, kernel_size=3, dropout=0.3):
        super(TCNEnc, self).__init__()
        self.seq_len = seq_len
        self.num_features = num_features
        self.num_channels = num_channels
        self.embedding_size = embedding_size
        self.kernel_size = kernel_size
        self.dropout = dropout

        self.tcn = TemporalConvNet(self.seq_len, num_channels=self.num_channels, kernel_size = self.kernel_size, dropout=self.dropout)
        self.linear = nn.Linear(self.num_channels[-1]*self.num_features, self.embedding_size)

    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        output = self.tcn(x)
        output = torch.flatten(output,start_dim=1)
        output = self.linear(output)
        return output

class TCNDec(nn.Module):
    def __init__(self, seq_len=50,num_features=36, num_channels=[20,20,20], embedding_size=128, kernel_size=3, dropout=0.3,encoder_tcn_out=4):
        super(TCNDec, self).__init__()
        self.seq_len = seq_len
        self.num_features = num_features
        self.num_channels = num_channels
        self.embedding_size = embedding_size
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.encoder_tcn_out =  encoder_tcn_out

        self.num_channels.append(seq_len)

        self.linear = nn.Linear(self.embedding_size,self.encoder_tcn_out*self.num_features)
        self.tcn = TemporalConvNet(self.encoder_tcn_out, num_channels=self.num_channels, kernel_size = self.kernel_size, dropout=self.dropout)

    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        output = self.linear(x)
        output = output.view(-1,self.encoder_tcn_out,self.num_features)
        output = self.tcn(output)
        return output

    
class TCNEncoderDecoder(nn.Module):
    def __init__(self, seq_len=50, num_features=36, num_channels=[20,20,20,4], embedding_size=128, kernel_size=3, dropout=0.3):
        super(TCNEncoderDecoder, self).__init__()
        self.seq_len = seq_len
        self.num_features = num_features
        self.num_channels_encoder = num_channels
        self.num_channels_decoder = num_channels[::-1][1:]
        self.embedding_size = embedding_size
        self.kernel_size = kernel_size
        self.dropout = dropout   

        self.encoder = TCNEnc(
            seq_len = self.seq_len,
            num_features = self.num_features,
            num_channels = self.num_channels_encoder,
            embedding_size = self.embedding_size,
            kernel_size = self.kernel_size,
            dropout = self.dropout)

        self.decoder = TCNDec(
            seq_len = self.seq_len,
            encoder_tcn_out= self.num_channels_encoder[-1],
            num_features = self.num_features,
            num_channels = self.num_channels_decoder,
            embedding_size = self.embedding_size,
            kernel_size = self.kernel_size,
            dropout = self.dropout)

    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        embedding = self.encoder(x)
        output = self.decoder(embedding)
        return output, embedding

In [6]:
ae_model = TCNEncoderDecoder(seq_len=50, num_features=36, num_channels=[40,40,40,30,20,4], embedding_size=128, kernel_size=3, dropout=0.3)
torchinfo.summary(ae_model, input_size=(32, 50, 36), col_names = ("input_size", "output_size", "num_params", "kernel_size", "mult_adds"))

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
TCNEncoderDecoder                             [32, 50, 36]              [32, 50, 36]              --                        3                         --
├─TCNEnc: 1-1                                 [32, 50, 36]              [32, 128]                 --                        3                         --
│    └─TemporalConvNet: 2-1                   [32, 50, 36]              [32, 4, 36]               --                        --                        --
│    │    └─Sequential: 3-1                   [32, 50, 36]              [32, 4, 36]               44,258                    --                        51,899,136
│    └─Linear: 2-2                            [32, 144]                 [32, 128]                 18,560                    --                        593,920
├─TCNDec: 1-2                                 [32, 128]       

In [7]:
prep_dir = './tmp/epoch150_randsample_input_l1.pt'
ae_model.load_state_dict(torch.load(prep_dir))


<All keys matched successfully>

---
Init Dataset

In [8]:
if config['device'] == 'cpu':
    device = "cpu"
else:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
skeleton_data = np.load('./data/skeleton_k10_v7_movements.npz')
skeleton_classes, skeleton_mov = skeleton_data['arr_0'], skeleton_data['arr_1']

In [11]:
label_map = [(1, 'lying'),
 (2, 'sitting'),
 (3, 'standing'),
 (4, 'walking'),
 (5, 'running'),
 (6, 'cycling'),
 (7, 'Nordic walking'),
 (9, 'watching TV'),
 (10, 'computer work'),
 (11, 'car driving'),
 (12, 'ascending stairs'),
 (13, 'descending stairs'),
 (16, 'vacuum cleaning'),
 (17, 'ironing'),
 (18, 'folding laundry'),
 (19, 'house cleaning'),
 (20, 'playing soccer'),
 (24, 'rope jumping')]

 
label2Id = {c[1]:i for i,c in enumerate(label_map)}
Id2Label = {i:c[1] for i,c in enumerate(label_map)}
action_dict = defaultdict(list)
skeleton_Ids = []
for i, a in enumerate(skeleton_classes):
    action_dict[label2Id[a]].append(i)
    skeleton_Ids.append(label2Id[a])

In [12]:
Id2Label

{0: 'lying',
 1: 'sitting',
 2: 'standing',
 3: 'walking',
 4: 'running',
 5: 'cycling',
 6: 'Nordic walking',
 7: 'watching TV',
 8: 'computer work',
 9: 'car driving',
 10: 'ascending stairs',
 11: 'descending stairs',
 12: 'vacuum cleaning',
 13: 'ironing',
 14: 'folding laundry',
 15: 'house cleaning',
 16: 'playing soccer',
 17: 'rope jumping'}

In [13]:
class SkeletonDataset(Dataset):
    def __init__(self, movements, actions, action_dict, seq_len=60):
        super(SkeletonDataset, self).__init__()
        self.movements = movements[:, :seq_len, ...]
        self.actions = actions
        self.action_dict = deepcopy(dict(action_dict))
        self.actionsIDs = list(self.action_dict.keys())
        

    def __getitem__(self, idx):
        x1 = self.movements[idx, ...]
        action = self.actions[idx]

        label = self.actionsIDs.index(action)
        # return np.transpose(x1, (1,0,2)), np.transpose(x2, (1,0,2)), label
        return x1, label

    def __len__(self):
        return self.movements.shape[0]

    def getShape(self):
        return self.movements[0, ...].shape


In [14]:
sample_dt = SkeletonDataset(skeleton_mov, skeleton_Ids, action_dict, seq_len=50)
sample_dl = DataLoader(sample_dt, batch_size=config['batch_size'], shuffle=True, pin_memory=True, drop_last=True)
for b in sample_dl:
    bx, by = b 
    # bx = torch.transpose(bx1, 1, 0)
    bs, seq_len, ft_in = bx.shape
    print(bx.shape)
    break 

torch.Size([16, 50, 36])


---
t-SNE before transfer learning

In [15]:
vector_pack = []
class_pack = []

ae_model = ae_model.to(device)

for x, y in sample_dl:
    _, vector = ae_model(x.float().to(device))
    vector_pack.append(vector.detach().cpu().numpy())
    class_pack.append(y.numpy())

In [16]:
all_vectors = np.concatenate(vector_pack)
all_actions = np.concatenate(class_pack)

In [17]:
all_vectors.shape, all_actions.shape

((176, 128), (176,))

In [34]:
tnse = TSNE(n_components=2, init='random', learning_rate='auto', perplexity=15, n_iter=1000)
cmp_data = tnse.fit_transform(all_vectors)

In [35]:
cmp_df = pd.DataFrame(data=cmp_data, columns=['x', 'y'])
cmp_df['label'] = all_actions
cmp_df['action'] = cmp_df['label'].map(Id2Label)

cmp_df.head()

,x,y,label,action
0,1.633858,-3.739381,14,folding laundry
1,10.531478,-18.969055,14,folding laundry
2,-8.868610,-10.061483,5,cycling
3,-23.589127,9.312789,8,computer work
4,14.413361,0.163391,16,playing soccer


In [20]:
def tsne_simulate(model, dl, class_map):
    vector_pack = []
    class_pack = []

    model = model.to(device)

    for x, y in dl:
        _, vector = model(x.float().to(device))
        vector_pack.append(vector.detach().cpu().numpy())
        class_pack.append(y.numpy())

    all_vectors = np.concatenate(vector_pack)
    all_actions = np.concatenate(class_pack)

    tnse = TSNE(n_components=2, init='random', learning_rate='auto', perplexity=15, n_iter=1000)
    cmp_data = tnse.fit_transform(all_vectors)

    cmp_df = pd.DataFrame(data=cmp_data, columns=['x', 'y'])
    cmp_df['label'] = all_actions
    cmp_df['action'] = cmp_df['label'].map(class_map)

    fig = px.scatter(cmp_df, x='x', y='y', color='action', width=800, height=800)
    fig.show()

In [45]:
def umap_simulate(model, dl, class_map):
    vector_pack = []
    class_pack = []

    model = model.to(device)

    for x, y in dl:
        _, vector = model(x.float().to(device))
        vector_pack.append(vector.detach().cpu().numpy())
        class_pack.append(y.numpy())

    all_vectors = np.concatenate(vector_pack)
    all_actions = np.concatenate(class_pack)

    tnse = UMAP(n_components=2, n_neighbors=5)
    cmp_data = tnse.fit_transform(all_vectors)

    cmp_df = pd.DataFrame(data=cmp_data, columns=['x', 'y'])
    cmp_df['label'] = all_actions
    cmp_df['action'] = cmp_df['label'].map(class_map)

    fig = px.scatter(cmp_df, x='x', y='y', color='action', width=800, height=800)
    fig.show()

In [36]:
px.scatter(cmp_df, x='x', y='y', color='action', width=800, height=800)


In [54]:
def umap_simulate(model, dl, class_map):
    vector_pack = []
    class_pack = []

    model = model.to(device)

    for x, y in dl:
        _, vector = model(x.float().to(device))
        vector_pack.append(vector.detach().cpu().numpy())
        class_pack.append(y.numpy())

    all_vectors = np.concatenate(vector_pack)
    all_actions = np.concatenate(class_pack)

    tnse = UMAP(n_components=2, n_neighbors=12)
    cmp_data = tnse.fit_transform(all_vectors)

    cmp_df = pd.DataFrame(data=cmp_data, columns=['x', 'y'])
    cmp_df['label'] = all_actions
    cmp_df['action'] = cmp_df['label'].map(class_map)

    fig = px.scatter(cmp_df, x='x', y='y', color='action', width=800, height=800)
    fig.show()

In [55]:
umap_simulate(ae_model, sample_dl, Id2Label)

--- 
transfer learn AE model - only by contrastive loss [AE-encoder]

In [24]:
def ae_train_step1(model , dataloader, optimizer, loss_module, device, batch_size, phase='train', l2_reg=False, loss_alpha=0.7):
    model = model.train()
    model.to(device)

    epoch_loss = 0 
    total_samples = 0 

    with tqdm(dataloader, unit='batch', desc=phase) as tepoch:
        for batch in tepoch:
            xa, labels = batch 
            xa = xa.float().to(device)
            labels = labels.float()

            # set optimizer grad to zero 
            optimizer.zero_grad()
            # get model prediction 
            with torch.set_grad_enabled(phase=='train'):
                skel_output, ft_output = model(xa)

            loss = loss_module['contrast'](ft_output, labels)

            if phase == 'train':
                loss.backward()
                optimizer.step()

            metrics = {'loss ': loss.item()}
            with torch.no_grad():
                    total_samples += len(labels)
                    epoch_loss += loss.item()
            
            tepoch.set_postfix(metrics)

    epoch_loss = epoch_loss/total_samples
    return metrics

In [25]:
skel_dt = SkeletonDataset(skeleton_mov, skeleton_Ids, action_dict, seq_len=config['seq_len'])
skel_dl = DataLoader(skel_dt, batch_size=config['batch_size'], shuffle=True, pin_memory=True, drop_last=True)

ae_model1 = TCNEncoderDecoder(seq_len=50, num_features=36, num_channels=[40,40,40,30,20,4], embedding_size=128, kernel_size=3, dropout=0.3)
prep_dir = './tmp/epoch150_randsample_input_l1.pt'
ae_model1.load_state_dict(torch.load(prep_dir))

optim = Adam(ae_model1.parameters(), lr=0.001)
ae_loss_module = {'contrast': ContrastiveLoss(pos_margin=0.1, neg_margin=0.8, distance=LpDistance())}
for epoch in tqdm(range(5), desc='Training Epoch', leave=False):
    ae_train_metrics = ae_train_step1(ae_model, skel_dl, optim, ae_loss_module, device, config['batch_size'], phase='train', loss_alpha=config['ae_alpha'])
    

train: 100%|██████████| 11/11 [00:00<00:00, 46.98batch/s, loss =0.665]


In [26]:
# view spatial distribution after transfer learning 
tsne_simulate(ae_model1, skel_dl, Id2Label)

c:\Users\deela\anaconda3\envs\mvts_trans\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

c:\Users\deela\anaconda3\envs\mvts_trans\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [46]:
umap_simulate(ae_model1, skel_dl, Id2Label)

---
train on SupCons loss function

In [34]:
class SkeletonDataset(Dataset):
    def __init__(self, movements, actions, action_dict, seq_len=60):
        super(SkeletonDataset, self).__init__()
        self.movements = movements[:, :seq_len, ...]
        self.actions = actions
        self.action_dict = deepcopy(dict(action_dict))
        self.actionsIDs = list(self.action_dict.keys())
        

    def __getitem__(self, idx):
        x1 = self.movements[idx, ...]
        action = self.actions[idx]

        partial_idx = random.sample(self.action_dict[action], k=1)[0]
        x2 = self.movements[partial_idx, ...]

        label = self.actionsIDs.index(action)
        # return np.transpose(x1, (1,0,2)), np.transpose(x2, (1,0,2)), label
        return x1, x2, label

    def __len__(self):
        return self.movements.shape[0]

    def getShape(self):
        return self.movements[0, ...].shape

In [67]:
from torchmetrics.functional import pairwise_euclidean_distance

In [70]:
def ae_train_step2(model , dataloader, optimizer, loss_module, device, batch_size, phase='train', l2_reg=False, loss_alpha=0.7):
    model = model.train()
    model.to(device)

    epoch_loss = 0 
    total_samples = 0 

    with tqdm(dataloader, unit='batch', desc=phase) as tepoch:
        for batch in tepoch:
            x1, x2, labels = batch 
            # post-process 
            xa = torch.vstack([x1, x2]) 
            xa = xa.float().to(device)
            labels = labels.float()

            # set optimizer grad to zero 
            optimizer.zero_grad()
            # get model prediction 
            with torch.set_grad_enabled(phase=='train'):
                skel_output, ft_output = model(xa)

            # reconstruct the output 
            f1, f2 = torch.split(ft_output, [batch_size//2, batch_size//2], dim=0)
            cons_output = torch.stack([f1.squeeze(1), f2.squeeze(1)], dim=1)
            # calc. contrastive loss 
            # print("mean distance > ", torch.diagonal(pairwise_euclidean_distance(f1, f2)).mean())
            loss = loss_module['contrast'](cons_output, labels)

            l2_lambda = 0.01
            l2_reg = torch.tensor(0.).to(device)

            for param in model.parameters():
                l2_reg += torch.norm(param)

            loss += l2_lambda * l2_reg

            if phase == 'train':
                loss.backward()
                optimizer.step()

            metrics = {'loss ': loss.item()}
            with torch.no_grad():
                    total_samples += len(labels)
                    epoch_loss += loss.item()
            
            tepoch.set_postfix(metrics)

    epoch_loss = epoch_loss/total_samples
    return metrics

In [71]:
skel_dt = SkeletonDataset(skeleton_mov, skeleton_Ids, action_dict, seq_len=config['seq_len'])
skel_dl = DataLoader(skel_dt, batch_size=config['batch_size']//2, shuffle=True, pin_memory=True, drop_last=True)

ae_model1 = TCNEncoderDecoder(seq_len=50, num_features=36, num_channels=[40,40,40,30,20,4], embedding_size=128, kernel_size=3, dropout=0.3)
prep_dir = './tmp/epoch150_randsample_input_l1.pt'
ae_model1.load_state_dict(torch.load(prep_dir))

optim = Adagrad(ae_model1.parameters(), lr=0.0001)
ae_loss_module = {'contrast': SupConLoss(temperature=0.07, contrast_mode='one')}
for epoch in tqdm(range(5), desc='Training Epoch', leave=False):
    ae_train_metrics = ae_train_step2(ae_model, skel_dl, optim, ae_loss_module, device, config['batch_size'], phase='train', loss_alpha=config['ae_alpha'])
    

Training Epoch:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\deela\anaconda3\envs\mvts_trans\lib\site-packages\torchmetrics\functional\pairwise\euclidean.py:34: UserWarning:

The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2985.)



mean distance >  tensor(3.7765, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(2.8992, device='cuda:0', grad_fn=<MeanBackward0>)


mean distance >  tensor(4.2843, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(5.1875, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(4.8355, device='cuda:0', grad_fn=<MeanBackward0>)


mean distance >  tensor(5.1309, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(6.2573, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(10.5736, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(3.4541, device='cuda:0', grad_fn=<MeanBackward0>)


mean distance >  tensor(4.6498, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(6.3178, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(2.7107, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(3.2321, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(5.4661, device='cuda:0', grad_fn=<MeanBackward0>)


...

train:  68%|██████▊   | 15/22 [00:00<00:00, 26.20batch/s, loss =nan]


mean distance >  tensor(3.3738, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(3.3369, device='cuda:0', grad_fn=<MeanBackward0>)


train:  82%|████████▏ | 18/22 [00:00<00:00, 27.21batch/s, loss =259]    

mean distance >  tensor(6.8642, device='cuda:0', grad_fn=<MeanBackward0>)
mean distance >  tensor(5.6718, device='cuda:0', grad_fn=<MeanBackward0>)


train: 100%|██████████| 22/22 [00:00<00:00, 26.08batch/s, loss =nan]


In [42]:
# view spatial distribution after transfer learning 
tsne_simulate(ae_model1, sample_dl, Id2Label)

c:\Users\deela\anaconda3\envs\mvts_trans\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

c:\Users\deela\anaconda3\envs\mvts_trans\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

